In [1]:
import pandas as pd
import numpy as np
 
#this cell is used for Data preparation

# import train and test data
train=pd.read_csv("D:/Downloads/dodgers_training.csv") 
test=pd.read_csv("D:/Downloads/dodgers_testing.csv")

train_y = train[["attend"]]
test_y =  test[["attend"]]

# add a new column in test and train
train["is_train"] = 1
test["is_train"] = 0

# combine test and train to make sure they have same number of columns
df = pd.concat([train, test])

# select only a few columns required for processing
df = df[["month","day_of_week","temp","skies","bobblehead","is_train"]]

# print (df)

# convert into categorical data  
df_month = pd.get_dummies(df['month'], prefix = 'month')
df = pd.concat([df, df_month], axis=1)
 
df_day_of_week = pd.get_dummies(df['day_of_week'], prefix = 'day_of_week')
df = pd.concat([df, df_day_of_week], axis=1)
 
df_skies = pd.get_dummies(df['skies'], prefix = 'skies')
df = pd.concat([df, df_skies], axis=1)

 
df_bobblehead = pd.get_dummies(df['bobblehead'], prefix = 'bobblehead')
df = pd.concat([df, df_bobblehead], axis=1)

# print (df)

#split test and train data

train_df = df[df["is_train"]==1]
test_df = df[df["is_train"]==0]
 
#drop the additional "is_train" and the columns which are converted via hot-encoding 
train_df.drop(columns=["is_train","month","day_of_week","skies","bobblehead"],inplace=True)
test_df.drop(columns=["is_train","month","day_of_week","skies","bobblehead"],inplace=True)
print(train_df)
print(test_df)



    temp  month_APR  month_AUG  month_JUL  month_JUN  month_MAY  month_OCT  \
0     80          0          1          0          0          0          0   
1     70          0          1          0          0          0          0   
2     80          0          1          0          0          0          0   
3     60          1          0          0          0          0          0   
4     84          0          1          0          0          0          0   
5     74          0          0          0          0          1          0   
6     67          0          0          0          0          1          0   
7     74          0          0          0          1          0          0   
8     73          0          0          1          0          0          0   
9     73          0          1          0          0          0          0   
10    70          0          0          0          0          1          0   
11    77          0          0          1          0          0 

D:\Python\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [2]:
#Question 1 # List the means and standard deviations of the encoded attributes before scaling.
data_columns = train_df.columns
#print(data_columns)
print ("Mean : \n" ,np.mean(train_df, axis = 0))
print ("\nStd dev: \n", np.std(train_df, axis = 0))

Mean : 
 temp                     73.642857
month_APR                 0.160714
month_AUG                 0.214286
month_JUL                 0.089286
month_JUN                 0.089286
month_MAY                 0.232143
month_OCT                 0.017857
month_SEP                 0.196429
day_of_week_Friday        0.178571
day_of_week_Monday        0.142857
day_of_week_Saturday      0.160714
day_of_week_Sunday        0.196429
day_of_week_Thursday      0.089286
day_of_week_Tuesday       0.089286
day_of_week_Wednesday     0.142857
skies_Clear               0.803571
skies_Cloudy              0.196429
bobblehead_NO             0.910714
bobblehead_YES            0.089286
dtype: float64

Std dev: 
 temp                     8.868944
month_APR                0.367267
month_AUG                0.410326
month_JUL                0.285156
month_JUN                0.285156
month_MAY                0.422200
month_OCT                0.132432
month_SEP                0.397296
day_of_week_Friday       0.

In [3]:
# Question 2 # What is the best L2 regularization coefficient? Provide the corresponding linear coefficients.

train_df = train_df.values.astype(float)
train_y = train_y.values.astype(float)

test_df = test_df.values.astype(float)
test_y = test_y.values.astype(float)

from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(train_df)
train_scaled = scaler.transform(train_df)
test_scaled = scaler.transform(test_df)

#What is the best L2 regularization coefficient? Provide the corresponding linear coefficients.
from sklearn.linear_model import RidgeCV
test_alphas = np.linspace(30,70,300)
#print (alphas)

ridge_cv = RidgeCV(alphas=test_alphas, store_cv_values=True)
ridge_cv.fit(train_scaled, train_y)

print ("L2 regularization coeff = ", ridge_cv.alpha_)
print ("Coefficients = ", ridge_cv.coef_)
print ("R^2 = ", ridge_cv.score(test_scaled, test_y))

#print(ridge_cv.alphas)
#print(ridge_cv.cv_values_)
#print(ridge_cv.cv_values_.shape)

L2 regularization coeff =  48.72909698996655
Coefficients =  [[  520.94542801  -277.38503477   462.356748    -215.72478307
   1121.37689144  -613.95905851   -81.13224568  -191.63671629
     70.35282744  -848.59546786   497.22529219   243.35562515
   -352.12040478  1086.96536753  -625.39358455   269.4464134
   -269.4464134  -1156.24837513  1156.24837513]]
R^2 =  0.3400003335616218


In [4]:
# Question 3 # What is the best L1 regularization coefficient? Provide the corresponding linear coefficients.
from sklearn.linear_model import LassoCV
lassocv = LassoCV(alphas= None, tol=0.009, cv=train_scaled.shape[1], max_iter=100000  )
lassocv.fit(train_scaled, train_y.flatten())
lasso_coeffs = lassocv.coef_
print ("L1 regularization coeff = ", lassocv.alpha_)
print ("Coefficients = ", lasso_coeffs)
print ("R^2 = ", lassocv.score(test_scaled, test_y))


L1 regularization coeff =  583.2435874914369
Coefficients =  [ 7.96972145e+02 -0.00000000e+00  6.83187647e+02 -0.00000000e+00
  1.82794430e+03 -7.52718463e+01 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -1.05021320e+03  1.49410868e+02  0.00000000e+00
 -3.11788874e+02  1.35246620e+03 -7.79869836e+02  0.00000000e+00
 -0.00000000e+00 -2.55027591e+03  8.44530795e-13]
R^2 =  0.31705906101892933


In [5]:
# Question 4 # What are the predictive attributes selected as a result of L1 regularization?

print("\n\nPredictive attributes selected:\n")
print("Predictive attributes are the ones with non-zero coefficients. Below are the predictive attributes and their co-efficient values:\n")

for i in range(data_columns.size):
    if(lasso_coeffs[i]!= 0):
        print ("\t",data_columns[i] , "\t: ", lasso_coeffs[i])
    



Predictive attributes selected:

Predictive attributes are the ones with non-zero coefficients. Below are the predictive attributes and their co-efficient values:

	 temp 	:  796.9721449610917
	 month_AUG 	:  683.1876471057183
	 month_JUN 	:  1827.9442957876784
	 month_MAY 	:  -75.27184628789647
	 day_of_week_Monday 	:  -1050.2131995730156
	 day_of_week_Saturday 	:  149.41086843944123
	 day_of_week_Thursday 	:  -311.7888741545525
	 day_of_week_Tuesday 	:  1352.4662042817408
	 day_of_week_Wednesday 	:  -779.8698362199912
	 bobblehead_NO 	:  -2550.275913979864
	 bobblehead_YES 	:  8.445307945034332e-13


In [6]:
''' Question 5 # For the models corresponding to the best L2 and L1 regularization coefficients list the following root-mean-square-error (RMSE): (1) RMSE on the training set, (2)
expected prediction RMSE obtained during the cross-validation, (3) RMSE on the testing
set. For the linear model without regularization list the following RMSE: (1) RMSE on
the training set, and (2) RMSE on the testing set.'''

from sklearn.metrics import mean_squared_error 
from math import sqrt

print("RMSE for L2: ")

l2_predicted_train = ridge_cv.predict(train_scaled)
RMSE_L2_train = sqrt(mean_squared_error(l2_predicted_train, train_y))
print ("\tTrain: ", RMSE_L2_train, "\n")
 
mean_l2 = np.matrix(ridge_cv.cv_values_.mean(axis=0))
se_l2= np.matrix(ridge_cv.cv_values_.std(axis=0)/np.sqrt(ridge_cv.cv_values_.shape[0]))
print("\tExpected Prediction: \n", np.sqrt(np.subtract(mean_l2,se_l2)), "\n")
#print ("size", np.sqrt(np.subtract(mean_l2,se_l2)).shape)

l2_predicted_test = ridge_cv.predict(test_scaled)
RMSE_L2_test = sqrt(mean_squared_error(l2_predicted_test, test_y))
print ("\tTest: ", RMSE_L2_test, "\n\n")

print("RMSE for L1: ")

l1_predicted_train = lassocv.predict(train_scaled)
RMSE_L1_train = sqrt(mean_squared_error(l1_predicted_train, train_y))
print ("\tTrain: ", RMSE_L1_train)

lassocv1 = LassoCV(alphas= np.array([lassocv.alpha_]), tol=0.009, cv=train_scaled.shape[1], max_iter=100000  )
lassocv1.fit(train_scaled, train_y.flatten())

l1_predicted_train_1 = lassocv1.predict(train_scaled)
RMSE_L1_train_1 = sqrt(mean_squared_error(l1_predicted_train_1, train_y))
print("\n\tExpected Prediction: \n",RMSE_L1_train_1 , "\n")

l1_predicted_test = lassocv.predict(test_scaled)
RMSE_L1_test = sqrt(mean_squared_error(l1_predicted_test, test_y))
print ("\tTest: ", RMSE_L1_test)

from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression(fit_intercept=True).fit(train_scaled, train_y)
linear_predicted_train = linear_reg.predict(train_scaled)
RMSE_LR_train = sqrt(mean_squared_error(linear_predicted_train, train_y))

print("\n\n RMSE for Linear Regression (without Regularization)\n\tTrain: ", RMSE_LR_train)

 
    
linear_predicted_test = linear_reg.predict(test_scaled)
RMSE_LR_test = sqrt(mean_squared_error(linear_predicted_test, test_y))

print("\tTest: ", RMSE_LR_test)



RMSE for L2: 
	Train:  5633.282044150267 

	Expected Prediction: 
 [[6029.5260047  6029.05605426 6028.5941046  6028.14008041 6027.69390721
  6027.25551136 6026.82482003 6026.40176121 6025.98626366 6025.57825695
  6025.17767142 6024.78443817 6024.39848904 6024.01975662 6023.64817426
  6023.28367599 6022.92619658 6022.57567151 6022.23203692 6021.89522967
  6021.56518729 6021.24184796 6020.92515054 6020.61503452 6020.31144005
  6020.01430789 6019.72357946 6019.43919676 6019.16110241 6018.88923966
  6018.6235523  6018.36398476 6018.110482   6017.86298959 6017.62145364
  6017.38582082 6017.15603836 6016.93205401 6016.71381609 6016.50127342
  6016.29437535 6016.09307175 6015.89731299 6015.70704996 6015.52223403
  6015.34281707 6015.16875143 6014.99998995 6014.83648592 6014.67819311
  6014.52506577 6014.37705857 6014.23412665 6014.0962256  6013.96331144
  6013.83534063 6013.71227005 6013.59405702 6013.48065925 6013.3720349
  6013.26814251 6013.16894104 6013.07438984 6012.98444867 6012.8990776

In [7]:
#Question 6 # What observations can you make based on the above RMSE?

print ("The test RMSE for L2 seem to have the lowest value. In my opinion, L2 is a better model than L1 and LR")


The test RMSE for L2 seem to have the lowest value. In my opinion, L2 is a better model than L1 and LR


In [8]:
# Question 7 # Predict the attendance on a clear Monday in June when the expected temperature is 72 for all three models with and without bobbleheads. Does bobblehead promotion have an impact on the attendance?  

#print (test_scaled.shape)

#print(data_columns)
''' 'temp', 'month_APR', 'month_AUG', 'month_JUL', 'month_JUN', 'month_MAY', 
       'month_OCT', 'month_SEP', 'day_of_week_Friday', 'day_of_week_Monday',
       'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday',
       'day_of_week_Tuesday', 'day_of_week_Wednesday', 'skies_Clear ',
       'skies_Cloudy', 'bobblehead_NO', 'bobblehead_YES'
       '''
without_bobblehead = np.array([[72, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0]])
with_bobblehead = np.array([[72, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1]])

#print(without_bobblehead.shape)
print ("Linear Regresssion:")
print ("\twithout bobblehead\t:" , linear_reg.predict(scaler.transform(without_bobblehead)))
print ("\twith bobblehead\t\t:" , linear_reg.predict(scaler.transform(with_bobblehead)))
 
print ("\nL1 model:")
print ("\twithout bobblehead\t:" , lassocv.predict(scaler.transform(without_bobblehead)))
print ("\twith bobblehead\t\t:" , lassocv.predict(scaler.transform(with_bobblehead)))
 
print ("\nL2 model:")
print ("\twithout bobblehead\t:" , ridge_cv.predict(scaler.transform(without_bobblehead)))
print ("\twith bobblehead\t\t:" , ridge_cv.predict(scaler.transform(with_bobblehead)))
print ("\nYes, the bobblehead prmotion has effect on attendance - the attendece predicted is higher in all the three models when there is bobblehead")

Linear Regresssion:
	without bobblehead	: [[41217.82779728]]
	with bobblehead		: [[51201.82779728]]

L1 model:
	without bobblehead	: [41595.90671667]
	with bobblehead		: [50539.35718397]

L2 model:
	without bobblehead	: [[40658.69618705]]
	with bobblehead		: [[48768.28936021]]

Yes, the bobblehead prmotion has effect on attendance - the attendece predicted is higher in all the three models when there is bobblehead
